# IoT data processing

In this lab we will demonstrate how to utilize Azure Machine Learning service to orchestrate TensorFlow training of IoT time series data.


## Scenario 

[Solar power forecasting](https://en.wikipedia.org/wiki/Solar_power_forecasting) is a challenging and important problem. The solar energy generation forecasting problem is closely linked to the problem of weather variables forecasting. Indeed, this problem is usually split into two parts, on one hand focusing on the forecasting of solar PV or any other meteorological variable and on the other hand estimating the amount of energy that a concrete power plant will produce with the estimated meteorological resource. In general, the way to deal with this difficult problem is usually related to the spatial and temporal scales we are interested in. This tutorial focusses on a simplified forecasting model using previously generated data from solar panel to predict the future. 

Using historic daily production of a solar panel, we want to predict the total power production of the solar panel array for a day. We will be using the LSTM based time series prediction model to predict the daily output of a solar panel based on the initial readings of the day. 

We train the model with historical data of the solar panel. In our example we want to predict the total power production of the solar panel array for the day starting with the initial readings of the day. We start predicting after the first 2 readings and adjust the prediction with each new reading.

## Data preparation

Our solar panel, emits two measures at every 30 min interval:
- `solar.current` is the current production in Watt
- `solar.total` is the total produced for the day so far 

Our prediction approach involves starting with the first 2 initial readings of the day. Based on these readings we start predicting and adjust the prediction with each new reading. The training data we are going to use comes as a csv file and has the following format:

>```
time,solar.current,solar.total
7am,6.3,1.7
7:30am,44.3,11.4
...
>```



In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/solar.csv')
df.head()

The training dataset we use contains data captured for 3 years.
The dataset is not pre-processed: it is raw data and contains smaller gaps and errors (like a panel failed to report).

To pre-process raw data we define a utility function `generate_solar_data()`. 

`generate_solar_data()` function performs the following tasks:
- read raw data into a pandas dataframe, 
- normalize the data, 
- groups by day and 
- append the columns "solar.current.max" and "solar.total.max", and
- generates the sequences for each day.

*Sequence generation*: All sequences are concatenated into a single list of sequences. There is *no more notion of timestamp* in our train input and **only** the sequences matter.

**Note** if we have less than 8 datapoints for a day we skip over the day assuming something is missing in the raw data. If we get more than 14 data points in a day we truncate the readings.

We split the dataset in the following manner: pick in sequence, 8 values for training, 1 for validation and 1 for test until there is no more data. This will spread training, validation and test datasets across the full timeline while preserving time order.


## Network model


We utilize LSTM network architecture for our model.

Corresponding  to the maximum possible 14 data points in the input sequence, we model our network with 14 LSTM cells, 1 cell for each data point we take during the day. 

The outputs from the LSTMs are fed into a dense layer and we randomly dropout 20% of the values to not overfit the model to the training set. The output of the dense layer becomes the prediction our model generates.

Notice: We lost the timestamp altogether; in our model only the sequences of readings matter. 

Our LSTM model has the following design:
![lstm](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/lstm.png)

The network model is an exact translation of the network diagram above.



## Training

### Connect to the workspace

In [ ]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

In [ ]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

### Upload the raw dataset to the default datastore

In [ ]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir='data', target_path='iot', overwrite=True, show_progress=True)

### Create AML Compute Target

Create AML Compute target with three GPU nodes.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os


# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 1)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 1)

vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_NC6")

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use the 'status' property    
    print(compute_target.status.serialize())

### Create a training script

In [ ]:
import os
script_folder = './script'
os.makedirs(script_folder, exist_ok=True)

In [ ]:
%%writefile $script_folder/train.py

import os
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.python.keras.layers import Add
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.models import Sequential

from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.estimator import model_to_estimator

def generate_solar_data(dataset_path, time_steps, normalize=1, val_size=0.1, test_size=0.1):
    """
    generate sequences to feed to rnn based on data frame with solar panel data
    the csv has the format: time ,solar.current, solar.total
     (solar.current is the current output in Watt, solar.total is the total production
      for the day so far in Watt hours)
    """
    
    df = pd.read_csv(dataset_path, index_col="time", parse_dates=['time'], dtype=np.float32)
    
    df["date"] = df.index.date
    
    # normalize data
    df['solar.current'] /= normalize
    df['solar.total'] /= normalize
    
    # group by day, find the max for a day and add a new column .max
    grouped = df.groupby(df.index.date).max()
    grouped.columns = ["solar.current.max", "solar.total.max", "date"]

    # merge continuous readings and daily max values into a single frame
    df_merged = pd.merge(df, grouped, right_index=True, on="date")
    df_merged = df_merged[["solar.current", "solar.total",
                           "solar.current.max", "solar.total.max"]]
    # we group by day so we can process a day at a time.
    grouped = df_merged.groupby(df_merged.index.date)
    per_day = []
    for _, group in grouped:
        per_day.append(group)

    # split the dataset into train, validatation and test sets on day boundaries
    val_size = int(len(per_day) * val_size)
    test_size = int(len(per_day) * test_size)
    next_val = 0
    next_test = 0

    result_x = {"train": [], "val": [], "test": []}
    result_y = {"train": [], "val": [], "test": []}    

    # generate sequences a day at a time
    for i, day in enumerate(per_day):
        # if we have less than 8 datapoints for a day we skip over the
        # day assuming something is missing in the raw data
        total = day["solar.total"].values.tolist()
        if len(total) < 8:
            continue
        if i >= next_val:
            current_set = "val"
            next_val = i + int(len(per_day) / val_size)
        elif i >= next_test:
            current_set = "test"
            next_test = i + int(len(per_day) / test_size)
        else:
            current_set = "train"
        max_total_for_day = np.array(day["solar.total.max"].values[0])
        for j in range(2, len(total)):
            result_x[current_set].append(total[0:j])
            result_y[current_set].append([max_total_for_day])
            if j >= time_steps:
                break
                
    # pad result_x sequences with 0 and convert to numpy.array
    # make result_y a numpy array
    for ds in ["train", "val", "test"]:
        sequences = pad_sequences(result_x[ds], dtype='float32')
        sequences = sequences.reshape((len(sequences), time_steps, 1))
        result_x[ds] = sequences
        result_y[ds] = np.array(result_y[ds])
    return result_x, result_y


def network_model(hidden_size, time_steps):
    """
    Define LSTM network model 
    """
    model = Sequential()
    model.add(LSTM(hidden_size, return_sequences=False, input_shape=(time_steps, 1)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    
    optimizer = 'rmsprop'
    loss = 'mse'
    metrics = ['mse']
    
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    return model


def input_fn(features, labels, train=False, batch_size=32, buffer_size=4000):
    """
    Input function to feed the data to the estimator
    """
   
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))    
    if train:
        dataset = dataset.shuffle(buffer_size)
        
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(-1)

    iterator = dataset.make_one_shot_iterator()
    X, y = iterator.get_next()
    
    return X, y


def main(argv=None):
       
    # Generate training and validation sequences
    # We keep up to 14 readings from a day
    TIMESTEPS = 14
    # 20000 is the maximum total output in our dataset. We normalize all values with this so our inputs are between 0.0 and 1.0 range
    NORMALIZE = 20000
    filepath = os.path.join(FLAGS.data_folder, FLAGS.filename)
    X, Y = generate_solar_data(filepath, TIMESTEPS, normalize=NORMALIZE)
    
    # Create the LSTM model
    lstm_model = network_model(FLAGS.hidden_size, TIMESTEPS)

    # Convert Keras model to TensorFlow estimator
    ckpt_folder = './logdir/' 
    estimator = model_to_estimator(keras_model = lstm_model, model_dir=ckpt_folder)

    # Create training and evaluation specifications
    train_input_fn = lambda: input_fn(X['train'], Y['train'], batch_size=FLAGS.batch_size, train=True)
    valid_input_fn = lambda: input_fn(X['val'], Y['val'], batch_size=FLAGS.batch_size, train=False)

    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, 
                                    max_steps=FLAGS.steps)
    
    eval_spec = tf.estimator.EvalSpec(input_fn=valid_input_fn, 
                                      steps=FLAGS.val_steps,
                                      exporters=None)

    # Start training
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

  
FLAGS = tf.app.flags.FLAGS

# Default global parameters
tf.app.flags.DEFINE_string('data_folder', 'iot', "Folder with training data")
tf.app.flags.DEFINE_string('filename', 'solar.csv', "Training file")
tf.app.flags.DEFINE_integer('steps', 1000, "Number of steps to train for")
tf.app.flags.DEFINE_integer('val_steps', 100, "Number of validation steps")
tf.app.flags.DEFINE_integer('hidden_size', 15, "Size of the hidden layer")
tf.app.flags.DEFINE_integer('batch_size', 32, "Batch size")

if __name__ == '__main__':
    tf.app.run()

### Create an Experiment

In [ ]:
from azureml.core import Experiment
experiment_name = 'distributed-training'
exp = Experiment(workspace=ws, name=experiment_name)

### Start and monitor the run

In [ ]:
from azureml.train.dnn import TensorFlow

ds = ws.get_default_datastore()

script_params = {
    '--data_folder': ds.path('iot').as_mount(),
    '--filename': 'solar.csv',
    '--steps': 10000,
    '--val_steps': 100
    
}

pip_packages = ['pandas==0.23.4']

est = TensorFlow(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                use_gpu=True,
                pip_packages=pip_packages)


In [ ]:
from azureml.widgets import RunDetails

tags = {"Compute target": "AML Compute GPU", "DNN": "LSTM"}
run = exp.submit(config=est, tags=tags)

RunDetails(run).show()